# Custom Types

## Defining data types

We can define types (i.e. data structures) ourselves using the `struct` keyword.

It is a convention that type names are capitalized and [camel cased](https://en.wikipedia.org/wiki/Camel_case).

(Note that types can not be redefined - you have to restart your Julia session to change a type definiton.)

In [1]:
struct MyType end

To create an object of type `MyType` we have to call a [constructor](https://docs.julialang.org/en/latest/manual/constructors/). Loosely speaking, a constructor is a function that create new objects.

Julia automatically creates a trivial constructors for us, which has the same name as the type.

In [2]:
methods(MyType)

# 1 method for type constructor:
[1] MyType() in Main at In[1]:1

In [3]:
m = MyType()

MyType()

In [4]:
typeof(m)

MyType

In [5]:
m isa MyType

true

Since no data is contained in our `MyType`  - it is a so-called *singleton type* - we can basically only use it for dispatch.

Most of the time, we'll want a self-defined type to hold some data. For this, we need *fields*.

In [6]:
struct A
    x::Int64
end

In [7]:
A()

MethodError: MethodError: no method matching A()
Closest candidates are:
  A(!Matched::Int64) at In[6]:2
  A(!Matched::Any) at In[6]:2

The default constructor always expects values for all fields.

In [8]:
A(3)

A(3)

In [9]:
a = A(3)

A(3)

In [10]:
# a.<TAB>
a.x

3

In [11]:
a.x

3

Note that types defined with `struct` are **immutable**, that is the values of it's fields cannot be changed.

In [12]:
a.x = 2

ErrorException: setfield! immutable struct of type A cannot be changed

In [13]:
mutable struct B
    x::Int64
end

In [14]:
b = B(3)

B(3)

In [15]:
b.x

3

In [16]:
b.x = 4

4

In [17]:
b.x

4

Note, however, that **immutability is not recursive**.

In [18]:
struct C
    x::Vector{Int64}
end

In [19]:
c = C([1, 2, 3])

C([1, 2, 3])

In [20]:
c.x

3-element Array{Int64,1}:
 1
 2
 3

In [21]:
c.x = [3,4,5]

ErrorException: setfield! immutable struct of type C cannot be changed

In [22]:
c.x[1] = 3

3

In [23]:
c.x

3-element Array{Int64,1}:
 3
 2
 3

In [ ]:
c.x .= [3,4,5] # dot to perform the assignment element-wise

In [25]:
g(x::Number) = x^2

g (generic function with 1 method)

In [26]:
y = rand(10)

10-element Array{Float64,1}:
 0.4400520537528818  
 0.027108545676754803
 0.9879177397098011  
 0.6005570470493597  
 0.8059143821728161  
 0.7776382953283703  
 0.062266814040939256
 0.8364837582677296  
 0.008853914473840874
 0.43866844158224705 

In [27]:
g.(y)

10-element Array{Float64,1}:
 0.1936458100121292   
 0.0007348732487087015
 0.9759814604333223   
 0.3606687667606469   
 0.6494979913929918   
 0.6047213183612136   
 0.0038771561308089102
 0.6997050778457055   
 7.839180151008892e-5 
 0.1924300016401973   

Abstract types are just as easy to define using the keyword `abstract type`.

In [28]:
abstract type MyAbstractType end

Since abstract types don't have fields, they only (informally) define interfaces and can be used for dispatch.

In [29]:
struct MyConcreteType <: MyAbstractType # subtype
    somefield::String
end

In [30]:
c = MyConcreteType("test")

MyConcreteType("test")

In [31]:
c isa MyAbstractType

true

In [32]:
supertype(MyConcreteType)

MyAbstractType

In [33]:
subtypes(MyAbstractType)

1-element Array{Any,1}:
 MyConcreteType

## Example: Diagonal Matrix

In [34]:
struct DiagonalMat
    diag::Vector{Float64}
end

In [35]:
DiagonalMat([1.2,4.3,5.0])

DiagonalMat([1.2, 4.3, 5.0])

### Arithmetic

In [36]:
import Base: +, -, *, /

+(Da::DiagonalMat, Db::DiagonalMat) = DiagonalMat(Da.diag + Db.diag)
-(Da::DiagonalMat, Db::DiagonalMat) = DiagonalMat(Da.diag - Db.diag)
*(Da::DiagonalMat, Db::DiagonalMat) = DiagonalMat(Da.diag .* Db.diag)
/(Da::DiagonalMat, Db::DiagonalMat) = DiagonalMat(Da.diag ./ Db.diag)

/ (generic function with 116 methods)

In [37]:
D1 = DiagonalMat([1,2,3])
D2 = DiagonalMat([2.4,1.9,5.7])

DiagonalMat([2.4, 1.9, 5.7])

In [38]:
D1 + D2

DiagonalMat([3.4, 3.9, 8.7])

In [39]:
D1 - D2

DiagonalMat([-1.4, 0.10000000000000009, -2.7])

In [40]:
D1 * D2

DiagonalMat([2.4, 3.8, 17.1])

In [41]:
D1 / D2

DiagonalMat([0.4166666666666667, 1.0526315789473684, 0.5263157894736842])

Arithmetics involving other types:

In [43]:
# Number
*(x::Number, D::DiagonalMat) = DiagonalMat(x * D.diag)
*(D::DiagonalMat, x::Number) = DiagonalMat(D.diag * x)
/(D::DiagonalMat, x::Number) = DiagonalMat(D.diag / x)

# Vector
*(D::DiagonalMat, V::AbstractVector) = D.diag .* V

* (generic function with 362 methods)

In [44]:
D1 * 2

DiagonalMat([2.0, 4.0, 6.0])

In [45]:
D1 * rand(3)

3-element Array{Float64,1}:
 0.04468796481553494
 1.2250879732987916 
 0.1786002099078181 

Note that some functions already work for our `DiagonalMat`:

In [46]:
sum([D1, D2])

DiagonalMat([3.4, 3.9, 8.7])

### Parameterization

In [47]:
DiagonalMat([1,2,3]) # implicit conversion to Vector{Float64}

DiagonalMat([1.0, 2.0, 3.0])

In [48]:
DiagonalMat([1+3im, 4-2im, im])

InexactError: InexactError: Float64(1 + 3im)

In [49]:
DiagonalMat(["Why", "not", "support", "strings?"])

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

We can easily relax our type definition to allow all sorts of internal value types.

In [50]:
struct DiagonalMatParam{T, V<:AbstractVector{T}}
    diag::V
end

# copied from above
import Base: +, -, *, /
+(Da::DiagonalMatParam, Db::DiagonalMatParam) = DiagonalMatParam(Da.diag + Db.diag)
-(Da::DiagonalMatParam, Db::DiagonalMatParam) = DiagonalMatParam(Da.diag - Db.diag)
*(Da::DiagonalMatParam, Db::DiagonalMatParam) = DiagonalMatParam(Da.diag .* Db.diag)
/(Da::DiagonalMatParam, Db::DiagonalMatParam) = DiagonalMatParam(Da.diag ./ Db.diag)
# Number
*(x::Number, D::DiagonalMatParam) = DiagonalMatParam(x * D.diag)
*(D::DiagonalMatParam, x::Number) = DiagonalMatParam(D.diag * x)
/(D::DiagonalMatParam, x::Number) = DiagonalMatParam(D.diag / x)
# Vector
*(D::DiagonalMatParam, V::AbstractVector) = D.diag .* V

* (generic function with 366 methods)

In [51]:
DiagonalMatParam([1+3im, 4-2im, im])

DiagonalMatParam{Complex{Int64},Array{Complex{Int64},1}}(Complex{Int64}[1 + 3im, 4 - 2im, 0 + 1im])

In [52]:
DiagonalMatParam(["This ", "just "]) * DiagonalMatParam(["should", "work!"])

DiagonalMatParam{String,Array{String,1}}(["This should", "just work!"])

### `AbstractArray` interface

Let's integrate our diagonal matrix into Julia's type hierarchy by subtyping `AbstractMatrix`. Of course, we should then also implement the [`AbstractArray` interface](https://docs.julialang.org/en/latest/manual/interfaces/#man-interface-array-1)!

In [89]:
struct DiagonalMatrix{T, V<:AbstractVector{T}} <: AbstractMatrix{T}
    diag::V
end

In [90]:
# implement AbstractArray interface
Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

function Base.getindex(D::DiagonalMatrix{T,V}, i::Int, j::Int) where {T,V}
    if i == j
        r = D.diag[i]
    else
        r = zero(T)
    end
    return r
end

function setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

setindex! (generic function with 1 method)

In [91]:
D = DiagonalMatrix([1,2,3])

3×3 DiagonalMatrix{Int64,Array{Int64,1}}:
 1  0  0
 0  2  0
 0  0  3

Note how it's automagically pretty printed!

In [58]:
D * D

3×3 Array{Int64,2}:
 1  0  0
 0  4  0
 0  0  9

In [59]:
D + D

3×3 Array{Int64,2}:
 2  0  0
 0  4  0
 0  0  6

In [60]:
D - D

3×3 Array{Int64,2}:
 0  0  0
 0  0  0
 0  0  0

In [61]:
D / D

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Basic arithmetics **just works!** What about broadcasting and more complicated functions?

In [62]:
sin.(D)

3×3 Array{Float64,2}:
 0.841471  0.0       0.0    
 0.0       0.909297  0.0    
 0.0       0.0       0.14112

In [63]:
sum([D, D, D])

3×3 Array{Int64,2}:
 3  0  0
 0  6  0
 0  0  9

In [64]:
using LinearAlgebra
eigen(D)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 1.0
 2.0
 3.0
eigenvectors:
3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

It is still advantageous to define fast versions that utilize the special diagonal structure:

In [65]:
@which D + D

+(A::AbstractArray, B::AbstractArray) in Base at arraymath.jl:38

In [97]:
+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)
*(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag .* Db.diag)
# Number
*(x::Number, D::DiagonalMatrix) = DiagonalMatrix(x * D.diag)
*(D::DiagonalMatrix, x::Number) = DiagonalMatrix(D.diag * x)
/(D::DiagonalMatrix, x::Number) = DiagonalMatrix(D.diag / x)

/ (generic function with 120 methods)

In [67]:
@which D + D

+(Da::DiagonalMatrix, Db::DiagonalMatrix) in Main at In[66]:1

An important thing to note is that **user defined types are just as good as built-in types**!

There is nothing special about built-in types. In fact, [they are implemented in precisely the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our `DiagonalMatrix` type does not come with any performance overhead by benchmarking it in a simple function.

# Benchmarking with `BenchmarkTools.jl`

In [68]:
+(3,5)

8

In [69]:
typeof(:x)

Symbol

In [70]:
⊗(x,y) = kron(x,y)

⊗ (generic function with 1 method)

In [71]:
rand(2,2) ⊗ rand(2,2)

4×4 Array{Float64,2}:
 0.162853  0.178675  0.316236  0.34696 
 0.206931  0.350451  0.401828  0.680521
 0.317694  0.34856   0.325042  0.356622
 0.403681  0.683659  0.413018  0.699471

In [73]:
using BenchmarkTools

In [98]:
operation(x) = x + 2*x

operation (generic function with 1 method)

In [78]:
x = rand(2,2)
@time operation.(x)

  0.000006 seconds (7 allocations: 304 bytes)


2×2 Array{Float64,2}:
 2.07496  2.25885
 2.36728  1.85431

In [76]:
function f()
    x = rand(2,2)
    @time operation.(x)
end

f (generic function with 2 methods)

In [77]:
f()

  0.000000 seconds (1 allocation: 112 bytes)


2×2 Array{Float64,2}:
 1.45204  2.10412 
 1.14705  0.676296

We should wrap benchmarks into functions!

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

In [79]:
@benchmark operation.(x)

BenchmarkTools.Trial: 
  memory estimate:  272 bytes
  allocs estimate:  7
  --------------
  minimum time:     219.760 ns (0.00% GC)
  median time:      229.234 ns (0.00% GC)
  mean time:        266.317 ns (6.91% GC)
  maximum time:     7.900 μs (96.37% GC)
  --------------
  samples:          10000
  evals/sample:     496

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [80]:
@btime operation.(x);

  220.381 ns (7 allocations: 272 bytes)


However, we still have to take some care to avoid accessing global variables.

In [84]:
a = "Hello"

"Hello"

In [85]:
b = "$a Oulu!"

"Hello Oulu!"

In [86]:
@btime operation.($x); # interpolate the value of x into the expression to avoid overhead of globals

  34.306 ns (1 allocation: 112 bytes)


More information: [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md).

Finally, we can check the performance of our custom volume type.

In [100]:
using LinearAlgebra
x = rand(100);
Djl = Diagonal(x)
D = DiagonalMatrix(x)
@btime operation(Djl);
@btime operation(D);

  156.663 ns (3 allocations: 1.77 KiB)
  156.452 ns (3 allocations: 1.77 KiB)


# Core messages of this Notebook

* **User defined types are as good as built-in types.**
* There are `mutable struct`s and immutable `struct`s and immutability is not recursive.
* We can easily **extend `Base` functions** for our types to implement arithmetics and such.
* **Subtyping an existing interface** can give lots of functionality for free.
* We should always benchmark our code with **BenchmarkTools.jl's @btime and @benchmark**.

# Exercise: One-hot vector

[One-hot encoding](https://en.wikipedia.org/wiki/One-hot) is useful in machine learning, as we'll see later.

It simply means that among a group of bits (all either 0 or 1) only one is hot (1) while all others are cold (0),

`v = [0, 0, 0, 0, 0, 1, 0, 0, 0]`

### Task

1. Think about what information an implementation of a one-hot vector actually has to store.
2. Define a `OneHot` type which represents a vector with only a single hot (i.e. `== 1`) bit.
3. Extend all the necessary `Base` functions such that the following computation works for a matrix `A` and a vector of `OneHot` vectors `vs` (i.e. `vs isa Vector{OneHot}`).

    ```julia
    function innersum(A, vs)
        t = zero(eltype(A)) # generic!
        for v in vs
            y = A*v
            for i in 1:length(vs[1])
                t += v[i] * y[i]
            end
        end
        return t
    end

    A = rand(3,3)
    vs = [rand(3) for _ = 1:10] # This should be replaced by a `Vector{OneHot}`

    innersum(A, vs)

    ```

4. Benchmark the speed of `innersum` when called with a `OneHot` vector or with a `Vector{Float64}`, respectively.
 * Do you observe a speed up?


5. Now, define a `OneHotVector` type which is identical to `OneHot` but is declared to be a subtype of `AbstractVector{Bool}` and extend only the functions `Base.getindex(v::OneHotVector, i::Int)` and `Base.size(v::OneHotVector)`.
 * Here, the function `size` should return a `Tuple{Int64}` indicating the length of the vector, i.e. `(3,)` for a one-hot vector of length 3 (see the [AbstractArray interface](https://docs.julialang.org/en/latest/manual/interfaces/#man-interface-array-1) for more information)
 

6. Try to create a single `OneHotVector` and try to run the `innersum` function using the new `OneHotVector` type.
 * What changes do you observe?
 * Do you have to implement any further methods?